<a href="https://colab.research.google.com/github/MarijanMarkovic/Numerical-modelling-of-differential-equations/blob/main/Laplace_vj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install -y --no-install-recommends fenics
!rm -rf *

In [ ]:
from fenics import*
import matplotlib.pyplot as plt


#1. Mesh Gen
n=10
mesh=UnitIntervalMesh(n)

#2. Definition of Finite Element Space
degree=1
V=FunctionSpace(mesh, 'CG', degree)

g_boundary=Constant(0.0)

def boundary(x, on_boundary):
  return on_boundary

bc=DirichletBC(V, g_boundary, boundary)

#3. Assembling/Solving
u=TrialFunction(V)
v=TestFunction(V)
a=dot(grad(u), grad(v)) * dx
f=Constant(1.0)
L=f * v * dx


u=Function(V)
solve(a==L, u, bc)

plt.show(u)
plot(u)

In [ ]:
from fenics import *

def laplace_solve(n, degree, f):
    # 1. mesh generation
    mesh = UnitIntervalMesh(n)

    # 2. definition of finite element space
    V = FunctionSpace(mesh, 'CG', degree)

    g_boundary = Constant(0.0)

    def boundary(x, on_boundary):
        return on_boundary

    bc = DirichletBC(V, g_boundary, boundary)

    # 3/4. assembling matrices and vectors/solve
    u = TrialFunction(V)
    v = TestFunction(V)

    a = dot(grad(u), grad(v)) * dx
    L = f * v * dx

    u = Function(V)
    solve(a == L, u, bc)

    return u


for degree in [1, 2]:
    f1 = Constant(1)
    u1 = Expression('x[0] * (1 - x[0]) / 2', degree=degree+2)

    f2 = Expression('sin(pi * x[0])', degree=degree+1)
    u2 = Expression('sin(pi * x[0]) / pi / pi', degree=degree+2)

    print('f = 1')
    print('------------------------------------------------')
    for n in [10, 20, 40, 80, 160]:
        uh = laplace_solve(n, degree, f1)

        eL2 = errornorm(u1, uh, 'L2')
        eH1 = errornorm(u1, uh, 'H10')

        print('n={} degree={} eL2={:.2e} eH1={:.2e}'.format(n, degree, eL2, eH1));
    print()

    print('f = sin(pi x)')
    print('------------------------------------------------')
    for n in [10, 20, 40, 80, 160]:
        uh = laplace_solve(n, degree, f2)

        eL2 = errornorm(u2, uh, 'L2')
        eH1 = errornorm(u2, uh, 'H10')

        print('n={} degree={} eL2={:.2e} eH1={:.2e}'.format(n, degree, eL2, eH1));
    print()
